### Converting Models via ONNX 
This script is designed to be the end-to-end master for converting models from one framework to another. The following frameworks are included:
* Keras
* Matlab (future)
* PyTorch
* Scikit-Learn
* TensorFlow

This scipt is to act as a function with the following parameters:
**Input(s):**
1. Model file name (provide absolute path if not within current dir)
2. 'frameworkIn' format
3. 'frameworkOut' format

**Output(s):**
1. Model in ONNX format
2. Model in 'frameworkOut' format

In [1]:
# User Inputs:
frameworkIn = "tensorflow"    # {keras | matlab | onnx | pytorch | scikit-learn | tensorflow}
frameworkOut = "keras"

#pathIn = "cifar_entire.pth"

netronView = False
verboseMode= True           # display amplifying information

In [2]:
# Imports
import pathlib
import onnx
import onnxruntime
import os
import tensorflow as tf

# Lesser packages that we want to track versions of
import numpy

print("---- 'ONNX_Master.ipynb' ---- \n")

print("Importing necessary packages... \n"
      "Required: {numpy | os | onnx | onnxruntime | pathlib | tensorflow}\n"
      "As needed: {keras | scikit-learn | tensorflow}\n")

print("Numpy version:\t\t", numpy.__version__, '\n'
      "ONNX version:\t\t", onnx.__version__, '\n'
      "ONNX Runtime version:\t", onnxruntime.__version__, '\n'
      "TensorFlow version:\t", tf.__version__, '\n')

---- 'ONNX_Master.ipynb' ---- 

Importing necessary packages... 
Required: {numpy | os | onnx | onnxruntime | pathlib | tensorflow}
As needed: {keras | scikit-learn | tensorflow}

Numpy version:		 1.19.2 
ONNX version:		 1.7.0 
ONNX Runtime version:	 1.5.2 
TensorFlow version:	 2.2.0 



In [3]:
# Debug helper - assumes all models are local 
if frameworkIn == "keras":
    pathIn = pathIn = "mnist-model.h5"
elif frameworkIn == "pytorch":
    pathIn = "vgg16.pth"
elif frameworkIn == "scikit-learn":
    pathIn = "iris-model.pkl"
elif frameworkIn == "tensorflow":
    pathIn = "MNIST_tf.pb"              # Not working
    pathIn = "mnist_tf.h5"
else:
    print("ERROR: Invalid framework!!")

**Defs for Generic Functions:**

In [4]:
# Change file extension
def ChangeExtension(fileIn, extOut):
    ind = fileIn.find('.') + 1
    base = fileIn[0:ind]
    fileOut = fileIn[0:ind] + extOut
    return fileOut, base

def GetExtension(fileIn):
    ind = fileIn.find('.') + 1
    ext = fileIn[ind:len(fileIn)]
    return ext

# Print saved status
def ModelSavedDialogue(modelSaved):
    print("Dir: ", os.getcwd())
    print("Model saved: ", modelSaved)

### Input File Handling

In [5]:
# Get import file
pathIn = pathlib.Path(pathIn)
modelIn = pathIn.name
modelONNX, baseName = ChangeExtension(modelIn, 'onnx') 

print("File path: ", pathIn, "\n\n"
      "Model in:\t", modelIn, "\n"
      "Model out:\t", modelONNX, "\n")

# Make sure modelIn is located within current working directory
import shutil

try:
    shutil.copy(pathIn, os.getcwd())
    print("'", modelIn, "' copied to current working directory...")
except shutil.SameFileError:
    pass

File path:  mnist_tf.h5 

Model in:	 mnist_tf.h5 
Model out:	 mnist_tf.onnx 



**Defs for Framework-In Conditions:**

In [6]:
# Keras
if frameworkIn == "keras" or frameworkOut == "keras":
    from tensorflow import keras
    from tensorflow.keras import layers
    import keras2onnx
    print("keras2onnx Version: "+ keras2onnx.__version__)

def Keras_ONNX(modelIn, modelONNX):
    # Load Keras model
    modelKeras = tf.keras.models.load_model(modelIn)
    print("Keras model loaded: ", modelIn)
    if verboseMode: 
        print("Displaying Keras model summary...\n")
        modelKeras.summary()
        print("-"*65, "\n", "-"*65)
    
    # Convert to ONNX
    debugMode = 0
    if verboseMode: 
        print("Displaying ONNX model summary...")
        debugMode = 1
    model = keras2onnx.convert_keras(modelKeras, baseName, debug_mode=debugMode)

    # Save the model in ONNX format
    keras2onnx.save_model(model, modelONNX)
    
    ModelSavedDialogue(modelONNX)

In [7]:
# Matlab


In [8]:
# PyTorch
if frameworkIn == "pytorch" or frameworkOut == "pytorch":
    import torch
    import torchvision

def PyTorch_ONNX(modelIn, modelONNX):
    # Load PyTorch model {.pt | .pth}
    model = torch.load(modelIn)
    print("PyTorch model loaded: ", modelIn)
    
    if verboseMode:
        from torchvision import models
        #from torchsummary import summary
        
        print("Displaying shape...\n", model)
        #summary(model, input_size=(1, 28, 28))
    
    # Convert to ONNX format   
    torch.onnx.export(model, 
                      torch.randn(1, 3, 224, 224), 
                      modelONNX)
    
    ModelSavedDialogue(modelONNX)
    

In [9]:
# Scikit-Learn
if frameworkIn == "scikit-learn" or frameworkOut == "scikit-learn":
    import pandas
    import pickle
    from skl2onnx import convert_sklearn
    from skl2onnx.common.data_types import FloatTensorType
    
def ScikitLearn_ONNX(modelIn, modelONNX):
    # Load SciKit-Learn Model (as .pkl)
    with open(modelIn, 'rb') as file:
        model = pickle.load(file)
        
    print("Scikit_Learn model loaded: ", modelIn)
    
    # Convert to ONNX
    initial_type = [('float_input', FloatTensorType([None, 4]))]
    onnx = convert_sklearn(model, initial_types=initial_type)
    with open(modelONNX, "wb") as file:
        file.write(onnx.SerializeToString())
    
    ModelSavedDialogue(modelONNX)

In [34]:
# TensorFlow
if frameworkIn == "tensorflow" or frameworkOut == "tensorflow":
#    import numpy # should already be imported
    import tf2onnx
    print("TF2ONNX Version: ", tf2onnx.__version__)
    
def TensorFlow_ONNX(modelIn, modelONNX):
    print("TensorFlow model loaded: ", modelIn)
    
    ext = GetExtension(modelIn)
    _, base = ChangeExtension(modelIn, "")
    modelDescription = base.join(" model")
    
    # Load TensorFlow model via one of two methods
    if ext == "pb":
    # Reference: https://docs.unity3d.com/Packages/com.unity.barracuda@1.0/manual/Exporting.html
        graph_def = tf.compat.v1.GraphDef()
        with open(modelIn, 'rb') as f:
            graph_def.ParseFromString(f.read())
        
        with tf.Graph().as_default() as graph:
            tf.import_graph_def(graph_def, name='')
        
        #inputs[:] = [i + ":0" for i in inputs]
        #outputs[:] = [o + ":0" for o in outputs]
    
        with tf.compat.v1.Session() as sess:
            g = tf2onnx.tfonnx.process_tf_graph(sess.graph, 
                                                input_names=inputs, 
                                                output_names=outputs)
            model_proto = g.make_model(modelDescription)
            checker = onnx.checker.check_model(model_proto)

            tf2onnx.utils.save_onnx_model(modelONNX, 
                                          feed_dict={}, 
                                          model_proto=model_proto)
    elif ext == "h5":
        tf.keras.models.load_model(modelIn) # expecting h5 format
    
        # Convert to ONNX
        with tf.compat.v1.Session() as sess:
            x = tf.compat.v1.placeholder(tf.float32, [2, 3], name="input")
            x_ = tf.compat.v1.add(x, x)
            _ = tf.compat.v1.identity(x_, name="output")

            # Convert Protobuf format and map to ONNX model
            onnx_graph = tf2onnx.tfonnx.process_tf_graph(sess.graph, 
                                                         input_names=["input:0"], 
                                                         output_names=["output:0"])
            model_proto = onnx_graph.make_model(modelDescription)
            with open(modelONNX, "wb") as f:
                f.write(model_proto.SerializeToString())
    
    ModelSavedDialogue(modelONNX)

TF2ONNX Version:  1.7.2


### Convert to ONNX

In [8]:
# Converting Class (frameworkIn -> ONNX)
print("Converting model: ", frameworkIn, " to ONNX...\n")

%run Conversion_To_ONNX.ipynb
print("Conversion_To_ONNX.ipynb running...")

# Run conversion script from external notebook
modelONNX = Convert_to_ONNX(modelIn, modelONNX, frameworkIn)

Converting model:  tensorflow  to ONNX...

---- Conversion_To_ONNX.ipynb ----
Base packages imported:	 onnx | onnxruntime | os
Conversion_To_ONNX.ipynb running...
tf2onnx:	 1.7.2 

Packages imported:	 {tf2onnx} 

TensorFlow model loaded:  mnist_tf.h5


NameError: name 'tf2onnx' is not defined

In [35]:
# Converting Class (frameworkIn -> ONNX)
print("Converting model: ", frameworkIn, " to ONNX...\n")

if frameworkIn == "keras":
    Keras_ONNX(modelIn, modelONNX)           # Tested
elif frameworkIn == "pytorch":
    PyTorch_ONNX(modelIn, modelONNX)         # Tested 
elif frameworkIn == "onnx":
    print("Model already in ONNX format!")
elif frameworkIn == "scikit-learn":
    ScikitLearn_ONNX(modelIn, modelONNX)     # Tested
elif frameworkIn == "tensorflow":
    TensorFlow_ONNX(modelIn, modelONNX)      # Tested (h5)
else:
    print("Invalid framework chosen.")


Converting model:  tensorflow  to ONNX...
TensorFlow model loaded:  mnist_tf.h5
Dir:  C:\Users\aaram\Desktop\Code 717\Machine Learning\ML Projects\ONNX
Model saved:  mnist_tf.onnx


### Load and Verify ONNX Model

In [36]:
# Load ONNX Model
onnxIn = onnx.load(modelONNX)

# Check that IR is well formed
onnx.checker.check_model(onnxIn)
print(modelONNX, " has been loaded and checked!")

# Print human readable representation of graph
#onnx.helper.printable_graph(modelONNX.graph)

# Use Netron to view model
if netronView:
    print("Loading Netron in separate tab...")
    netron.start(onnxIn, port=8081)

mnist_tf.onnx  has been loaded and checked!


**Defs for Converting ONNX to frameworkOut**

In [ ]:
# Keras [Output]


In [1]:
#if frameworkOut == "pytorch":
    
def ONNX_PyTorch(onnxIn, modelOut):
    print("Currently PyTorch does not support conversion from ONNX: ", "\n"
          "https://stackoverflow.com/questions/58833870/cant-we-run-an-onnx-model-imported-to-pytorch#:~:text=PyTorch%20doesn't%20currently%20support,to%20and%20from%20various%20frameworks.")
          
    # Convert to PyTorch
    #torch.save(model, modelOut)

In [15]:
if frameworkOut == "tensorflow":
    from onnx_tf.backend import prepare

def ONNX_TensorFlow(onnxIn):
    # ONNX model already loaded and verified above [onnxIn]
    
    # Import ONNX model to TensorFlow
    tf_rep = prepare(onnxIn)
    tf_rep.export_graph(modelOut)
    
    modelLoad = modelBase + "\saved_model.pb"
    ModelSavedDialogue(modelLoad)
    
    # Display results
    print("Displaying model...", "\n"
          "Input nodes: ", tf_rep.inputs, "\n"
          "Model output nodes: ", tf_rep.outputs, "\n\n"
          "All Model nodes: ", tf_rep.tensor_dict)
   

### Convert to FrameworkOut

In [39]:
# Converting Class (ONNX -> frameworkOut)
if frameworkOut == "keras": 
    modelOut = ChangeExtension(onnxIn, "h5")       # 
    ONNX_keras(onnxIn, modelOut)
elif frameworkOut == "onnx":                     
    print("ONNX file already created!")
elif frameworkOut == "pytorch": 
    modelOut = ChangeExtension(onnxIn, "pt")       #
    ONNX_PyTorch(onnxIn, modelOut)
elif frameworkOut == "scikit-learn": 
    modelOut = ChangeExtension(onnxIn, "pb")       #
    ONNX_ScikitLearn(onnxIn, modelOut), 
elif frameworkOut == "tensorflow": 
    modelOut = ChangeExtension(onnxIn, "tf")       #
    ONNX_TensorFlow(onnxIn, modelOut),
    
print("Conversion complete!")
print("-"*65)

ONNX file already created!
Conversion complete!
-----------------------------------------------------------------
